In [3]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [6]:
import pandas as pd
df=pd.read_csv('../data/consolidado_dataset.csv', encoding='latin1')
df.head(10)

,SEGMENTO,PERIODO,PAIS,CLASIFICACION_COMENTARIOS,COMENTARIO,EMPRESA
0,PYMES,202408,ARGENTINA,7,el servicio me funciona muy bien no hay duda u...,Telefonica
1,PYMES,202408,ARGENTINA,10,siempre he tenido esa calidad es de primera pa...,Telefonica
2,PYMES,202408,ARGENTINA,8,en cuanto al tema del seguro no me han dado un...,Telefonica
3,PYMES,202408,ARGENTINA,7,por el servicio tecnico cuando tuve problemas ...,Telefonica
4,PYMES,202408,ARGENTINA,5,no tenemos buena señal.,Telefonica
5,PYMES,202408,ARGENTINA,8,porque comparado con otras empresas tienen muc...,Telefonica
6,PYMES,202408,ARGENTINA,1,porque nunca me solucionan los problemas se co...,Telefonica
7,PYMES,202408,ARGENTINA,6,porque cuesta mucho comunicarse con un asesor ...,Telefonica
8,PYMES,202408,ARGENTINA,6,por el servicio al cliente es malo cuando hago...,Telefonica
9,PYMES,202408,ARGENTINA,7,si es por empresas tenemos conflictos con los ...,Telefonica


1-Preprocesamiento y análisis de sentimiento

In [9]:
import spacy
import re

# Cargar modelo en español de spaCy
nlp = spacy.load("es_core_news_sm")

# Función para limpiar y lematizar texto
def preprocesar_texto(texto):
    # Eliminar caracteres especiales y URLs
    texto = re.sub(r"http\S+|www\S+|https\S+", '', texto, flags=re.MULTILINE)
    texto = re.sub(r'\@w+|\#','', texto)
    texto = re.sub(r'[^A-Za-z\s]', '', texto)
    
    # Aplicar tokenización y lematización
    doc = nlp(texto)
    tokens = [token.lemma_ for token in doc if not token.is_stop]
    
    return " ".join(tokens)

# Aplicar preprocesamiento al dataset
df['comentario_preprocesado'] = df['COMENTARIO'].apply(preprocesar_texto)


In [10]:
df.head(10)

,SEGMENTO,PERIODO,PAIS,CLASIFICACION_COMENTARIOS,COMENTARIO,EMPRESA,comentario_preprocesado
0,PYMES,202408,ARGENTINA,7,el servicio me funciona muy bien no hay duda u...,Telefonica,servicio funcionar duda buscar servicio funcio...
1,PYMES,202408,ARGENTINA,10,siempre he tenido esa calidad es de primera pa...,Telefonica,calidad
2,PYMES,202408,ARGENTINA,8,en cuanto al tema del seguro no me han dado un...,Telefonica,tema seguro solucin robar tel junio repona...
3,PYMES,202408,ARGENTINA,7,por el servicio tecnico cuando tuve problemas ...,Telefonica,servicio tecnico tener problema internet demor...
4,PYMES,202408,ARGENTINA,5,no tenemos buena señal.,Telefonica,seal
5,PYMES,202408,ARGENTINA,8,porque comparado con otras empresas tienen muc...,Telefonica,comparado empresa facilidad comunicarno wsp ...
6,PYMES,202408,ARGENTINA,1,porque nunca me solucionan los problemas se co...,Telefonica,solucionar problema cortar internet cambiar mo...
7,PYMES,202408,ARGENTINA,6,porque cuesta mucho comunicarse con un asesor ...,Telefonica,costar comunicar él asesor quisir cambiar plan...
8,PYMES,202408,ARGENTINA,6,por el servicio al cliente es malo cuando hago...,Telefonica,servicio cliente malo reclamo flota reposición...
9,PYMES,202408,ARGENTINA,7,si es por empresas tenemos conflictos con los ...,Telefonica,empresa conflicto pago tener q cargar él por...


In [11]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# Cargar modelo y tokenizador de BERT para análisis de sentimientos
tokenizer = AutoTokenizer.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")
model = AutoModelForSequenceClassification.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")

# Función para predecir sentimientos
def obtener_sentimiento(texto):
    inputs = tokenizer(texto, return_tensors="pt", truncation=True, padding=True)
    outputs = model(**inputs)
    # Obtener la predicción de sentimiento
    predicciones = torch.softmax(outputs.logits, dim=-1)
    sentimiento = torch.argmax(predicciones).item()
    
    # Mapear los sentimientos
    sentimiento_map = {0: "Muy negativo", 1: "Negativo", 2: "Neutral", 3: "Positivo", 4: "Muy positivo"}
    return sentimiento_map[sentimiento]

# Aplicar análisis de sentimiento al texto preprocesado
df['sentimiento'] = df['comentario_preprocesado'].apply(obtener_sentimiento)

# Guardar el DataFrame con la nueva columna 'sentimiento' en un archivo CSV
df.to_csv("../data/df_sentimientos.csv", index=False, encoding='utf-8')


C:\Users\jfioni\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\jfioni\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\jfioni\.cache\huggingface\hub\models--nlptown--bert-base-multilingual-uncased-sentiment. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://

2- Entrenamiento del modelo

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Convertir los textos preprocesados a vectores TF-IDF
vectorizer = TfidfVectorizer(max_features=5000)
X = vectorizer.fit_transform(df['comentario_preprocesado'])

# Definir las etiquetas para clasificación (ejemplo: países, segmentos, etc.)
y = df['SEGMENTO']  # Ajustar a la columna que queremos clasificar


2.1- Entrenar modelo de clasificación

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# Dividir los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenar un modelo de Random Forest
clf = RandomForestClassifier()
clf.fit(X_train, y_train)

# Evaluar el modelo
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))


3- Visualización del modelo con TensorBoard

In [ ]:
from torch.utils.tensorboard import SummaryWriter

# Inicializar TensorBoard
writer = SummaryWriter()

# Almacenar métricas y visualizar
for epoch in range(epochs):
    # Aquí iría el proceso de entrenamiento...
    writer.add_scalar('Accuracy/train', accuracy, epoch)
    writer.add_scalar('Loss/train', loss, epoch)

# Al finalizar
writer.close()


4- Guardar y comparar modelos

In [ ]:
import joblib

# Guardar el modelo
joblib.dump(clf, '../models/modelo_clasificacion.pkl')

# Cargar el modelo en el futuro
clf_cargado = joblib.load('../models/modelo_clasificacion.pkl')
